In [3]:
import json

In [4]:
def load_json(file):
    f = open(file, 'r')
    return json.load(f)

In [5]:
def write_json(data):
    encoded_data = json.JSONEncoder(indent=2).encode(data)
    
    f = open('mud_of_babel_data.json', 'w')
    f.write(encoded_data)

In [6]:
def build_basic_game_info():
    return {
        'commandCounter' : 0,
        'gameOver' : False,
        'introText' : 'Welcome to the MUD of Babel',
        'outroText' : 'Thanks For playing!',
        'player' : {
            'currentLocation' : '',
            'inventory' : {},
            'lightSource' : False
        },
        'map': {}
    }

In [9]:
def build_room(parent, info, next_room=None):
    _id = "%s_%s" % (parent, info['id'])
    _subject = info['subject']
    _child_count = len(info['children'])
    exit_south = 'A_AC'
    if (next_room != None):
        exit_south = list(next_room.keys())[0]
    room = {
        _id: {
            'firstVisit' : True,
            'displayName' : _subject,
            'description' : "The description for %s\n\n" % _subject,
#             'setup' : 'function(){end();}',
            'interactables' : {
                'helmets' : { 'look' : 'It is a pile of miner helmets with lights on them. They seem to still be operational.' },
                'sign' : { 'look' : 'The sign reads "Crooked Gulch Gold Mine" and has a note tacked to the bottom of it.' },
                'note' : { 'look' : 'Written in an untidy scroll the note reads "Generator blew. Lights out."' }
            },
            'items' : {
                'helmet' : {
                    'displayName' : 'Miner Helmet',
                    'description' : 'A trusty old miner helmet covered in minor dents. Still seems sturdy and the light works.',
#                     'use' : 'function(){return useLightSource();}',
                    'quantity' : 1,
                    'hidden' : True
                }
            },
            'exits' : {
                'south' : {
                    'displayName' : 'South',
                    'destination' : exit_south
                }
            }
        }
    }
    return room

In [10]:
def parse_class_file(class_code):
    raw_json = load_json(("%s.json" % class_code))
    subject = raw_json[class_code]['subject']
    game_info = build_basic_game_info()
    rooms = {}
    previous_room = None
    first_room = None
    for key, value in raw_json[class_code]['children'].items():
        room = build_room(parent=class_code, info=value, next_room=previous_room)
        rooms.update(room)
        previous_room = room
        first_room = list(room.keys())[0]
    game_info['map'] = rooms
    game_info['player']['currentLocation'] = first_room
    write_json(game_info)

parse_class_file('A')